### Importación de librerías

In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Definición de variables

In [2]:
DATASET_PATH = "../../dataset/gestos_cara"
MODEL_FILE = "modelo_gestos_cara.pkl"

### Lógica de entreno

In [4]:
# Configuración de MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh

# static_image_mode=True: Optimizado para procesar fotos sueltas (más preciso)
# refine_landmarks=True: Añade puntos del iris (clave para detectar ojos cerrados)
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

data = []
labels = []

# Mapeo de carpetas a números (Debe coincidir con tus carpetas)
mapa_etiquetas = {
    "0_Neutro": 0,
    "1_Ojos_Cerrados": 1,
    "2_Cabeza_Der": 2,
    "3_Cabeza_Izq": 3
}

print(f"--- ENTRENADOR FACIAL ---")
print(f"Buscando imágenes en: {DATASET_PATH}...")

# 1. BUCLE DE LECTURA DE IMÁGENES
for nombre_carpeta, etiqueta_num in mapa_etiquetas.items():
    ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
    
    if not os.path.exists(ruta_carpeta):
        print(f"AVISO: No existe la carpeta {nombre_carpeta}. Saltando...")
        continue
        
    archivos = os.listdir(ruta_carpeta)
    print(f"Procesando '{nombre_carpeta}': {len(archivos)} fotos found.")
    
    count_ok = 0
    count_fail = 0
    
    for archivo in archivos:
        # Filtrar solo imágenes
        if not (archivo.endswith(".jpg") or archivo.endswith(".png")):
            continue
            
        ruta_img = os.path.join(ruta_carpeta, archivo)
        img = cv2.imread(ruta_img)
        
        if img is None:
            continue

        # MediaPipe necesita RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Procesar la cara
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Extraemos los 478 puntos (x, y)
                landmark_list = []
                for lm in face_landmarks.landmark:
                    landmark_list.append(lm.x)
                    landmark_list.append(lm.y)
                    # No usamos Z por ahora para simplificar, X e Y bastan para gestos
                
                data.append(landmark_list)
                labels.append(etiqueta_num)
                count_ok += 1
        else:
            count_fail += 1

    print(f"  -> {count_ok} caras detectadas | {count_fail} fallos/sin cara")

# 2. ENTRENAMIENTO
if len(data) > 0:
    print("\n--- ENTRENANDO INTELIGENCIA ARTIFICIAL ---")
    
    # Convertir listas a arrays de Numpy (más rápido)
    data = np.array(data)
    labels = np.array(labels)

    # Separar 20% de los datos para prueba (Test) y 80% para entrenar (Train)
    # Esto nos dice qué tan bueno es el modelo antes de guardarlo
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

    model = RandomForestClassifier()
    
    # Entrenar con el 80%
    model.fit(x_train, y_train)

    # Probar con el 20% restante
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Precisión estimada del modelo: {accuracy * 100:.2f}%")
    
    if accuracy < 0.7:
        print("ADVERTENCIA: La precisión es baja. Intenta tomar mejores fotos o más cantidad.")

    # Re-entrenar con EL 100% DE LOS DATOS para el archivo final
    print("Generando modelo final con todos los datos...")
    model.fit(data, labels)
    
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"¡LISTO! Modelo guardado como: '{MODEL_FILE}'")
    print("Ya puedes usar este archivo en tu juego.")

else:
    print("\nERROR CRÍTICO: No se encontraron datos válidos.")
    print("Asegúrate de haber ejecutado 'recolector_cara.py' primero y tener fotos en las carpetas.")


--- ENTRENADOR FACIAL ---
Buscando imágenes en: ../../dataset/gestos_cara...
Procesando '0_Neutro': 330 fotos found.


c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


  -> 330 caras detectadas | 0 fallos/sin cara
Procesando '1_Ojos_Cerrados': 334 fotos found.
  -> 329 caras detectadas | 5 fallos/sin cara
Procesando '2_Cabeza_Der': 300 fotos found.
  -> 198 caras detectadas | 102 fallos/sin cara
Procesando '3_Cabeza_Izq': 300 fotos found.
  -> 159 caras detectadas | 141 fallos/sin cara

--- ENTRENANDO INTELIGENCIA ARTIFICIAL ---
Precisión estimada del modelo: 100.00%
Generando modelo final con todos los datos...
¡LISTO! Modelo guardado como: 'modelo_gestos_cara.pkl'
Ya puedes usar este archivo en tu juego.


In [6]:
import os
import cv2
import numpy as np
import pickle
import mediapipe as mp
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

# --- CONFIGURACIÓN ---
DATASET_PATH = "../../dataset/gestos_cara" # <--- CAMBIA ESTO
MODEL_FILE = "modelo_facial_red_augmentation.pkl"

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

mapa_etiquetas = {
    "0_Neutro": 0,
    "1_Ojos_Cerrados": 1,
    "2_Cabeza_Der": 2,
    "3_Cabeza_Izq": 3
}

# --- FUNCIÓN DE NORMALIZACIÓN (Igual que antes) ---
def normalizar_puntos(landmarks):
    coords = np.array([[lm.x, lm.y] for lm in landmarks])
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    max_dist = np.max(np.abs(centered))
    if max_dist > 0:
        normalized = centered / max_dist
    else:
        normalized = centered
    return normalized.flatten()

# --- NUEVA FUNCIÓN: DATA AUGMENTATION ---
def aumentar_datos(vector_original):
    """
    Genera variaciones sintéticas de los puntos originales.
    Entrada: vector 1D de puntos normalizados.
    Salida: Lista de vectores (original + variaciones).
    """
    variaciones = []
    
    # 1. El original siempre se guarda
    variaciones.append(vector_original)
    
    # Reconstruimos la forma (478, 2) para operar geométricamente
    puntos_2d = vector_original.reshape(-1, 2)

    # --- TÉCNICA A: RUIDO ALEATORIO (Jitter) ---
    # Añadimos un ruido muy pequeño (0.01 = 1% de variación)
    ruido = np.random.normal(0, 0.01, puntos_2d.shape)
    puntos_ruido = puntos_2d + ruido
    variaciones.append(puntos_ruido.flatten())

    # --- TÉCNICA B: ESCALADO LEVE ---
    # Hacemos la cara un 5% más pequeña o más grande
    factor_escala = np.random.uniform(0.95, 1.05)
    puntos_escala = puntos_2d * factor_escala
    variaciones.append(puntos_escala.flatten())
    
    # --- TÉCNICA C: ROTACIÓN LEVE (Opcional pero recomendada) ---
    # Rotamos +/- 10 grados. (Matemática de rotación 2D básica)
    theta = np.radians(np.random.uniform(-10, 10))
    c, s = np.cos(theta), np.sin(theta)
    matriz_rotacion = np.array(((c, -s), (s, c)))
    
    # Aplicar rotación: puntos * matriz
    # Nota: .dot funciona bien porque los puntos ya están centrados en (0,0) por la normalización
    puntos_rotados = np.dot(puntos_2d, matriz_rotacion)
    variaciones.append(puntos_rotados.flatten())

    return variaciones

# --- 1. RECOLECCIÓN DE DATOS CON AUMENTO ---
data = []
labels = []

print("--- PROCESANDO IMÁGENES + DATA AUGMENTATION ---")

for nombre_carpeta, etiqueta_num in mapa_etiquetas.items():
    ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
    if not os.path.exists(ruta_carpeta): continue
    
    archivos = os.listdir(ruta_carpeta)
    print(f"Leyendo clase '{nombre_carpeta}'...")
    
    for archivo in archivos:
        if not (archivo.endswith(".jpg") or archivo.endswith(".png")): continue
        
        ruta_img = os.path.join(ruta_carpeta, archivo)
        img = cv2.imread(ruta_img)
        if img is None: continue
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 1. Normalizamos
                vector_base = normalizar_puntos(face_landmarks.landmark)
                
                # 2. Aumentamos (Generamos 3 copias extra por cada foto)
                vectores_aumentados = aumentar_datos(vector_base)
                
                # 3. Guardamos todos
                for v in vectores_aumentados:
                    data.append(v)
                    labels.append(etiqueta_num)

data = np.array(data)
labels = np.array(labels)

# --- 2. ENTRENAMIENTO DEEP LEARNING (Tema 5) ---
if len(data) > 0:
    print(f"\nDatos totales (Originales + Aumentados): {len(data)} muestras.")
    print("Entrenando Red Neuronal (MLP)...")

    # Aumentamos un poco las iteraciones ya que tenemos más datos
    model = MLPClassifier(
        hidden_layer_sizes=(128, 64), 
        activation='relu', 
        solver='adam', 
        max_iter=1500,  # Más iteraciones
        random_state=42
    )

    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, data, labels, cv=kfold)
    
    print(f"\nResultados Validación Cruzada:")
    print(f"PRECISIÓN MEDIA: {scores.mean() * 100:.2f}% (+/- {scores.std() * 100:.2f}%)")

    model.fit(data, labels)
    
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    print(f"Modelo guardado en {MODEL_FILE}")

else:
    print("No se encontraron datos.")

--- PROCESANDO IMÁGENES + DATA AUGMENTATION ---
Leyendo clase '0_Neutro'...


c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Leyendo clase '1_Ojos_Cerrados'...
Leyendo clase '2_Cabeza_Der'...
Leyendo clase '3_Cabeza_Izq'...

Datos totales (Originales + Aumentados): 4064 muestras.
Entrenando Red Neuronal (MLP)...

Resultados Validación Cruzada:
PRECISIÓN MEDIA: 98.52% (+/- 0.49%)
Modelo guardado en modelo_facial_red_augmentation.pkl


In [8]:
import os
import cv2
import mediapipe as mp

# --- CONFIGURACIÓN ---
DATASET_PATH = "../../dataset/gestos_cara"  # <--- PON TU RUTA AQUÍ

# Configuración de MediaPipe (Igual que en el entrenamiento)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

def auditar_dataset():
    print(f"--- AUDITORÍA DE DATASET: {DATASET_PATH} ---\n")
    
    if not os.path.exists(DATASET_PATH):
        print("ERROR: La ruta del dataset no existe.")
        return

    carpetas = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
    
    total_global_imgs = 0
    total_global_ok = 0
    total_global_fail = 0
    archivos_fallidos = []

    for carpeta in carpetas:
        ruta_carpeta = os.path.join(DATASET_PATH, carpeta)
        archivos = os.listdir(ruta_carpeta)
        
        # Filtrar solo imágenes
        imagenes = [f for f in archivos if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        count_ok = 0
        count_fail = 0
        local_fails = []

        print(f"📂 Revisando carpeta: '{carpeta}' ({len(imagenes)} imágenes)")

        for archivo in imagenes:
            ruta_img = os.path.join(ruta_carpeta, archivo)
            img = cv2.imread(ruta_img)

            if img is None:
                print(f"   [ERROR DE LECTURA] {archivo}")
                count_fail += 1
                local_fails.append(archivo)
                continue

            # Convertir a RGB para MediaPipe
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(img_rgb)

            if results.multi_face_landmarks:
                count_ok += 1
            else:
                count_fail += 1
                local_fails.append(archivo)
                # Guardamos la ruta completa para el reporte final
                archivos_fallidos.append(os.path.join(carpeta, archivo))

        # Estadísticas de la carpeta
        if len(imagenes) > 0:
            tasa = (count_ok / len(imagenes)) * 100
        else:
            tasa = 0
            
        print(f"   ✅ Detectadas: {count_ok}")
        print(f"   ❌ Fallidas:   {count_fail}")
        print(f"   📊 Tasa de éxito: {tasa:.1f}%")
        
        if local_fails:
            print(f"   ⚠️ Archivos problemáticos en esta carpeta: {local_fails}")
        print("-" * 30)

        total_global_imgs += len(imagenes)
        total_global_ok += count_ok
        total_global_fail += count_fail

    # --- REPORTE FINAL ---
    print("\n" + "="*40)
    print("       RESULTADO FINAL DE LA AUDITORÍA")
    print("="*40)
    print(f"TOTAL IMÁGENES:      {total_global_imgs}")
    print(f"TOTAL CARAS VÁLIDAS: {total_global_ok}")
    print(f"TOTAL FALLOS:        {total_global_fail}")
    
    if total_global_imgs > 0:
        tasa_global = (total_global_ok / total_global_imgs) * 100
        print(f"EFICIENCIA DEL DATASET: {tasa_global:.2f}%")
    
    if archivos_fallidos:
        print("\n--- LISTA NEGRA (Archivos que deberías borrar o revisar) ---")
        for f in archivos_fallidos:
            print(f" -> {f}")
    else:
        print("\n¡Excelente! Tu dataset está 100% limpio.")

if __name__ == "__main__":
    auditar_dataset()

--- AUDITORÍA DE DATASET: ../../dataset/gestos_cara ---

📂 Revisando carpeta: '0_Neutro' (330 imágenes)


c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


   ✅ Detectadas: 330
   ❌ Fallidas:   0
   📊 Tasa de éxito: 100.0%
------------------------------
📂 Revisando carpeta: '1_Ojos_Cerrados' (334 imágenes)
   ✅ Detectadas: 329
   ❌ Fallidas:   5
   📊 Tasa de éxito: 98.5%
   ⚠️ Archivos problemáticos en esta carpeta: ['1_Ojos_Cerrados_Kaggle_168.jpg', '1_Ojos_Cerrados_Kaggle_17.jpg', '1_Ojos_Cerrados_Kaggle_175.jpg', '1_Ojos_Cerrados_Kaggle_18.jpg', '1_Ojos_Cerrados_Kaggle_251.jpg']
------------------------------
📂 Revisando carpeta: '2_Cabeza_Der' (300 imágenes)
   ✅ Detectadas: 198
   ❌ Fallidas:   102
   📊 Tasa de éxito: 66.0%
   ⚠️ Archivos problemáticos en esta carpeta: ['2_Cabeza_Der_Kaggle_100.jpg', '2_Cabeza_Der_Kaggle_103.jpg', '2_Cabeza_Der_Kaggle_106.jpg', '2_Cabeza_Der_Kaggle_107.jpg', '2_Cabeza_Der_Kaggle_108.jpg', '2_Cabeza_Der_Kaggle_113.jpg', '2_Cabeza_Der_Kaggle_116.jpg', '2_Cabeza_Der_Kaggle_120.jpg', '2_Cabeza_Der_Kaggle_122.jpg', '2_Cabeza_Der_Kaggle_124.jpg', '2_Cabeza_Der_Kaggle_127.jpg', '2_Cabeza_Der_Kaggle_129.jpg'